<a href="https://www.kaggle.com/code/yussufkadir/sales-forecast?scriptVersionId=224482086" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


# 1. Importing Libraries that is going to be used for the data preprocessing and visualizing.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.checkpoint import checkpoint

# 2. Data exploration and preprocessing.

In [3]:
submission_sample = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv")
submission_sample.head()

,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,0.0
4,3000892,0.0


#### This cell is suggesting that target is sales column.

#### Data needs to be read first and we will parse the dates when the data contains date because we need to ensure that data is in date time format.

### 2.2 Turning all data into one piece.

In [4]:
train = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv", parse_dates=['date'])
test = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv", parse_dates=['date'])
holiday_events = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv", parse_dates=['date'])
transactions = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/transactions.csv", parse_dates=['date'])
stores = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/stores.csv")
oil = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/oil.csv", parse_dates=['date'])

In [5]:
df = train.merge(stores, on='store_nbr', how='left')
df = df.merge(oil, on='date', how='left')

holiday_events['is_holiday'] = 1
holiday_events = holiday_events[['date','is_holiday']].drop_duplicates(subset='date')

df = df.merge(holiday_events, on='date', how='left')

df = df.merge(transactions, on=['date', 'store_nbr'], how='left')

df.fillna(0, inplace=True)
df.set_index('date', inplace=True)

df.head()

,id,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,is_holiday,transactions
date,,,,,,,,,,,,
2013-01-01,0,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,0.0,1.0,0.0
2013-01-01,1,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,0.0,1.0,0.0
2013-01-01,2,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,0.0,1.0,0.0
2013-01-01,3,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,0.0,1.0,0.0
2013-01-01,4,1,BOOKS,0.0,0,Quito,Pichincha,D,13,0.0,1.0,0.0


In [6]:
df['weekdays'] = df.index.dayofweek
df['month'] = df.index.month
df['year'] = df.index.year

df['is_weekend'] = (df.index.dayofweek > 5).astype(int)
df.head()

,id,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,is_holiday,transactions,weekdays,month,year,is_weekend
date,,,,,,,,,,,,,,,,
2013-01-01,0,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,0.0,1.0,0.0,1,1,2013,0
2013-01-01,1,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,0.0,1.0,0.0,1,1,2013,0
2013-01-01,2,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,0.0,1.0,0.0,1,1,2013,0
2013-01-01,3,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,0.0,1.0,0.0,1,1,2013,0
2013-01-01,4,1,BOOKS,0.0,0,Quito,Pichincha,D,13,0.0,1.0,0.0,1,1,2013,0


In [7]:
label_encoder = LabelEncoder()
label_encoded_cols = ['family','city','state','type']
df['family'] = label_encoder.fit_transform(df['family'])
df['city'] = label_encoder.fit_transform(df['city'])
df['state'] = label_encoder.fit_transform(df['state'])
df['type'] = label_encoder.fit_transform(df['type'])

df.head()

,id,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,is_holiday,transactions,weekdays,month,year,is_weekend
date,,,,,,,,,,,,,,,,
2013-01-01,0,1,0,0.0,0,18,12,3,13,0.0,1.0,0.0,1,1,2013,0
2013-01-01,1,1,1,0.0,0,18,12,3,13,0.0,1.0,0.0,1,1,2013,0
2013-01-01,2,1,2,0.0,0,18,12,3,13,0.0,1.0,0.0,1,1,2013,0
2013-01-01,3,1,3,0.0,0,18,12,3,13,0.0,1.0,0.0,1,1,2013,0
2013-01-01,4,1,4,0.0,0,18,12,3,13,0.0,1.0,0.0,1,1,2013,0


In [8]:
df['lag_7'] = df['sales'].shift(7)
df['window_7_mean'] = df['sales'].rolling(window=7).mean()
df.fillna(0, inplace=True)
df.head()

,id,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,is_holiday,transactions,weekdays,month,year,is_weekend,lag_7,window_7_mean
date,,,,,,,,,,,,,,,,,,
2013-01-01,0,1,0,0.0,0,18,12,3,13,0.0,1.0,0.0,1,1,2013,0,0.0,0.0
2013-01-01,1,1,1,0.0,0,18,12,3,13,0.0,1.0,0.0,1,1,2013,0,0.0,0.0
2013-01-01,2,1,2,0.0,0,18,12,3,13,0.0,1.0,0.0,1,1,2013,0,0.0,0.0
2013-01-01,3,1,3,0.0,0,18,12,3,13,0.0,1.0,0.0,1,1,2013,0,0.0,0.0
2013-01-01,4,1,4,0.0,0,18,12,3,13,0.0,1.0,0.0,1,1,2013,0,0.0,0.0


In [9]:
test = test.merge(oil, on='date', how='left')
test = test.merge(stores, on='store_nbr', how='left')

label_encoder = LabelEncoder()
label_encoded_cols = ['family','city','state','type']
test['family'] = label_encoder.fit_transform(test['family'])
test['city'] = label_encoder.fit_transform(test['city'])
test['state'] = label_encoder.fit_transform(test['state'])
test['type'] = label_encoder.fit_transform(test['type'])

test.fillna(0)
test.head()

,id,date,store_nbr,family,onpromotion,dcoilwtico,city,state,type,cluster
0,3000888,2017-08-16,1,0,0,46.8,18,12,3,13
1,3000889,2017-08-16,1,1,0,46.8,18,12,3,13
2,3000890,2017-08-16,1,2,2,46.8,18,12,3,13
3,3000891,2017-08-16,1,3,20,46.8,18,12,3,13
4,3000892,2017-08-16,1,4,0,46.8,18,12,3,13


In [10]:
test.set_index('date', inplace=True)

test['weekdays'] = test.index.dayofweek
test['month'] = test.index.month
test['year'] = test.index.year

test['is_weekend'] = (test.index.dayofweek > 5).astype(int)
test.head()

,id,store_nbr,family,onpromotion,dcoilwtico,city,state,type,cluster,weekdays,month,year,is_weekend
date,,,,,,,,,,,,,
2017-08-16,3000888,1,0,0,46.8,18,12,3,13,2,8,2017,0
2017-08-16,3000889,1,1,0,46.8,18,12,3,13,2,8,2017,0
2017-08-16,3000890,1,2,2,46.8,18,12,3,13,2,8,2017,0
2017-08-16,3000891,1,3,20,46.8,18,12,3,13,2,8,2017,0
2017-08-16,3000892,1,4,0,46.8,18,12,3,13,2,8,2017,0


In [11]:
df.drop(['is_holiday', 'transactions', 'lag_7', 'window_7_mean'], axis=1, inplace=True)
df.head()

,id,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,weekdays,month,year,is_weekend
date,,,,,,,,,,,,,,
2013-01-01,0,1,0,0.0,0,18,12,3,13,0.0,1,1,2013,0
2013-01-01,1,1,1,0.0,0,18,12,3,13,0.0,1,1,2013,0
2013-01-01,2,1,2,0.0,0,18,12,3,13,0.0,1,1,2013,0
2013-01-01,3,1,3,0.0,0,18,12,3,13,0.0,1,1,2013,0
2013-01-01,4,1,4,0.0,0,18,12,3,13,0.0,1,1,2013,0


In [12]:
scaler = StandardScaler()
df[['onpromotion', 'dcoilwtico']] = scaler.fit_transform(df[['onpromotion', 'dcoilwtico']])
df.head()
test[['onpromotion', 'dcoilwtico']] = scaler.transform(test[['onpromotion', 'dcoilwtico']])
test.head()

,id,store_nbr,family,onpromotion,dcoilwtico,city,state,type,cluster,weekdays,month,year,is_weekend
date,,,,,,,,,,,,,
2017-08-16,3000888,1,0,-0.213012,-0.002914,18,12,3,13,2,8,2017,0
2017-08-16,3000889,1,1,-0.213012,-0.002914,18,12,3,13,2,8,2017,0
2017-08-16,3000890,1,2,-0.049331,-0.002914,18,12,3,13,2,8,2017,0
2017-08-16,3000891,1,3,1.423799,-0.002914,18,12,3,13,2,8,2017,0
2017-08-16,3000892,1,4,-0.213012,-0.002914,18,12,3,13,2,8,2017,0


In [13]:
X = df.drop('sales',axis=1)
y = df['sales']

X = np.array(X)
y = np.array(y)

In [14]:
def sequence_creator(data, target, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:(i + seq_length)]
        y = target[i + seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

In [15]:
seq_length = 7
X_seq, y_seq = sequence_creator(X, y, seq_length)

In [16]:
split_factor = int(0.8 * len(X))
X_train = X_seq[:split_factor]
y_train = y_seq[:split_factor]
X_test = X_seq[split_factor:]
y_test = y_seq[split_factor:]

In [17]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [18]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    def forward(self,x):
        print(f"Type of x: {type(x)}")
        print(f"Shape attribute exists: {hasattr(x, 'shape')}")
        print(f"Size method exists: {hasattr(x, 'size')}")
    
        if isinstance(x, torch.Tensor):
            print(f"x shape: {x.shape}")
        else:
            print("x is not a torch.Tensor")
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = checkpoint(self.lstm, x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out
        
input_size = X_train_tensor.shape[2]
hidden_size = 50
num_layers = 2
output_size = 1

model = LSTMModel(input_size, hidden_size, num_layers, output_size)

In [19]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
y_train_tensor = y_train_tensor.reshape(-1,1)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 100000
batch_size2 = 50000
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size2, shuffle=False)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    batch_count = 0

    for X_batch, y_batch in train_loader:
        
        optimizer.zero_grad()

        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        batch_count += 1
    avg_loss = total_loss / batch_count
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

Type of x: <class 'torch.Tensor'>
Shape attribute exists: True
Size method exists: True
x shape: torch.Size([100000, 7, 13])


/opt/conda/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Type of x: <class 'torch.Tensor'>
Shape attribute exists: True
Size method exists: True
x shape: torch.Size([100000, 7, 13])
Type of x: <class 'torch.Tensor'>
Shape attribute exists: True
Size method exists: True
x shape: torch.Size([100000, 7, 13])
Type of x: <class 'torch.Tensor'>
Shape attribute exists: True
Size method exists: True
x shape: torch.Size([100000, 7, 13])
Type of x: <class 'torch.Tensor'>
Shape attribute exists: True
Size method exists: True
x shape: torch.Size([100000, 7, 13])
Type of x: <class 'torch.Tensor'>
Shape attribute exists: True
Size method exists: True
x shape: torch.Size([100000, 7, 13])
Type of x: <class 'torch.Tensor'>
Shape attribute exists: True
Size method exists: True
x shape: torch.Size([100000, 7, 13])
Type of x: <class 'torch.Tensor'>
Shape attribute exists: True
Size method exists: True
x shape: torch.Size([100000, 7, 13])
Type of x: <class 'torch.Tensor'>
Shape attribute exists: True
Size method exists: True
x shape: torch.Size([100000, 7, 13])


In [20]:
model.eval()
total_test_loss = 0
batch_count = 0

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        total_test_loss += loss.item()
        batch_count += 1

avg_test_loss = total_test_loss / batch_count
print(f"Test loss: {avg_test_loss:.4f}")

Type of x: <class 'torch.Tensor'>
Shape attribute exists: True
Size method exists: True
x shape: torch.Size([50000, 7, 13])


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([50000])) that is different to the input size (torch.Size([50000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Type of x: <class 'torch.Tensor'>
Shape attribute exists: True
Size method exists: True
x shape: torch.Size([50000, 7, 13])
Type of x: <class 'torch.Tensor'>
Shape attribute exists: True
Size method exists: True
x shape: torch.Size([50000, 7, 13])
Type of x: <class 'torch.Tensor'>
Shape attribute exists: True
Size method exists: True
x shape: torch.Size([50000, 7, 13])
Type of x: <class 'torch.Tensor'>
Shape attribute exists: True
Size method exists: True
x shape: torch.Size([50000, 7, 13])
Type of x: <class 'torch.Tensor'>
Shape attribute exists: True
Size method exists: True
x shape: torch.Size([50000, 7, 13])
Type of x: <class 'torch.Tensor'>
Shape attribute exists: True
Size method exists: True
x shape: torch.Size([50000, 7, 13])
Type of x: <class 'torch.Tensor'>
Shape attribute exists: True
Size method exists: True
x shape: torch.Size([50000, 7, 13])
Type of x: <class 'torch.Tensor'>
Shape attribute exists: True
Size method exists: True
x shape: torch.Size([50000, 7, 13])
Type of 

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([171])) that is different to the input size (torch.Size([171, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
